In [ ]:
pip install pycaret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 320 kB 6.9 MB/s 
     |████████████████████████████████| 167 kB 54.8 MB/s 
     |████████████████████████████████| 1.7 MB 45.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 88 kB 4.7 MB/s 
     |████████████████████████████████| 16.5 MB 310 kB/s 
     |████████████████████████████████| 56 kB 3.1 MB/s 
     |████████████████████████████████| 3.3 MB 49.8 MB/s 
     |████████████████████████████████| 325 kB 51.2 MB/s 
     |████████████████████████████████| 2.0 MB 46.4 MB/s 
     |████████████████████████████████| 1.3 MB 48.3 MB/s 
     |████████████████████████████████| 662 kB 53.6 MB/s 
     |████████████████████████████████| 141 kB 52.6 MB/s 
     |████████████████████████████████| 6

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

In [ ]:
df_train = pd.read_csv('/content/train_data.csv')

In [ ]:
df_train['Date'] =pd.to_datetime(df_train['Date'])

In [ ]:
df_train.head()

,Date,Petrol (USD)
0,2003-06-09,74.59
1,2003-06-16,74.47
2,2003-06-23,74.42
3,2003-06-30,74.35
4,2003-07-07,74.28


In [ ]:
df_train.rename(columns={df_train.columns[1]:'USD'},inplace=True)

In [ ]:
df_train['12-month-avg'] = df_train['USD'].rolling(12).mean()

In [ ]:
import plotly.express as px
fig = px.line(df_train,x='Date',y=['USD','12-month-avg'],template = 'plotly_dark')
fig.show()

In [ ]:
df_train['month'] = [i.month for i in df_train['Date']]
df_train['year'] = [i.year for i in df_train['Date']]
df_train['week'] = [i.week for i in df_train['Date']]
df_train['Series'] = np.arange(1,len(df_train)+1)

In [ ]:
df_train.drop(['12-month-avg','Date'],axis=1,inplace=True)
df_train = df_train[['Series','year','month','week','USD']]

In [ ]:
df_train.head()

,Series,year,month,week,USD
0,1,2003,6,24,74.59
1,2,2003,6,25,74.47
2,3,2003,6,26,74.42
3,4,2003,6,27,74.35
4,5,2003,7,28,74.28


In [ ]:
train = df_train[df_train['year']<2018]
test = df_train[df_train['year']>=2018]

In [ ]:
train.shape,test.shape

((760, 5), (53, 5))

In [ ]:
from pycaret.regression import *

In [ ]:
s = setup(data =train,test_data =test,target='USD',fold_strategy = 'timeseries',numeric_features = ['year', 'Series'],fold = 3, transform_target = True, session_id = 123)

,Description,Value
0,session_id,123
1,Target,USD
2,Original Data,"(760, 5)"
3,Missing Values,True
4,Numeric Features,3
5,Categorical Features,1
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(758, 14)"


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=[], ml_usecase='regression',
                                      numerical_features=['year', 'Series'],
                                      target='USD', time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numer...
                ('scaling', 'passthrough'), ('P_transform', 'passthrough'),
                ('binn', 'passthrough'), ('rem_outliers', 'passthrough'),
                ('cluster_all', 'passthrough'),
             

In [ ]:
best = compare_models(sort = 'MAE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,14.9150,278.6949,16.6745,-5.7470,0.1447,0.1262,0.2200
dt,Decision Tree Regressor,15.4646,335.8461,18.2370,-6.3477,0.1532,0.1318,0.0233
gbr,Gradient Boosting Regressor,15.7710,305.8409,17.4707,-6.1030,0.1528,0.1339,0.0567
lightgbm,Light Gradient Boosting Machine,15.9754,313.2480,17.6150,-6.3339,0.1511,0.1355,0.2800
et,Extra Trees Regressor,16.1120,317.5053,17.7652,-6.1380,0.1565,0.1373,0.1833
knn,K Neighbors Regressor,17.7311,382.7877,19.3428,-9.2365,0.1671,0.1487,0.0300
par,Passive Aggressive Regressor,17.8231,513.2194,19.3619,-17.0905,0.1720,0.1429,1.0100
ada,AdaBoost Regressor,18.5254,442.7487,20.1369,-12.5471,0.1751,0.1531,0.0633
huber,Huber Regressor,23.4541,764.2233,24.9623,-23.8155,0.2306,0.1907,0.0367
llar,Lasso Least Angle Regression,23.7609,777.3559,25.2580,-24.0994,0.2336,0.1933,0.9967


INFO:logs:create_model_container: 12
INFO:logs:master_model_container: 12
INFO:logs:display_container: 2
INFO:logs:PowerTransformedTargetRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                                max_depth=None, max_features='auto',
                                max_leaf_nodes=None, max_samples=None,
                                min_impurity_decrease=0.0,
                                min_impurity_split=None, min_samples_leaf=1,
                                min_samples_split=2,
                                min_weight_fraction_leaf=0.0, n_estimators=100,
                                n_jobs=-1, oob_score=False,
                                power_transformer_method='box-cox',
                                power...
                                regressor=RandomForestRegressor(bootstrap=True,
                                                                ccp_alpha=0.0,
                                                                crit

In [ ]:
prediction_holdout = predict_model(best);

INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=PowerTransformedTargetRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                                max_depth=None, max_features='auto',
                                max_leaf_nodes=None, max_samples=None,
                                min_impurity_decrease=0.0,
                                min_impurity_split=None, min_samples_leaf=1,
                                min_samples_split=2,
                                min_weight_fraction_leaf=0.0, n_estimators=100,
                                n_jobs=-1, oob_score=False,
                                power_transformer_method='box-cox',
                                power...
                                regressor=RandomForestRegressor(bootstrap=True,
                                                                ccp_alpha=0.0,
                                                                criterion='mse',
                             

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Random Forest Regressor,7.5794,84.5679,9.1961,-4.0275,0.0746,0.0594


In [ ]:
# generate predictions on the original dataset**
predictions = predict_model(best, data=df_train)

# add a date column in the dataset**
predictions['Date'] = pd.date_range(start='2003-06-09', end = '2018-12-31', freq = 'W-MON')

# line plot**
fig = px.line(predictions, x='Date', y=["USD", "Label"], template = 'plotly_dark')

# add a vertical rectange for test-set separation**
fig.add_vrect(x0="2018-01-01", x1="2018-12-31", fillcolor="grey", opacity=0.25, line_width=0)
fig.show()

INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=PowerTransformedTargetRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                                max_depth=None, max_features='auto',
                                max_leaf_nodes=None, max_samples=None,
                                min_impurity_decrease=0.0,
                                min_impurity_split=None, min_samples_leaf=1,
                                min_samples_split=2,
                                min_weight_fraction_leaf=0.0, n_estimators=100,
                                n_jobs=-1, oob_score=False,
                                power_transformer_method='box-cox',
                                power...
                                regressor=RandomForestRegressor(bootstrap=True,
                                                                ccp_alpha=0.0,
                                                                criterion='mse',
                             

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Random Forest Regressor,0,0,0,0,0,0


In [ ]:
final_best = finalize_model(best)

INFO:logs:Initializing finalize_model()
INFO:logs:finalize_model(estimator=PowerTransformedTargetRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                                max_depth=None, max_features='auto',
                                max_leaf_nodes=None, max_samples=None,
                                min_impurity_decrease=0.0,
                                min_impurity_split=None, min_samples_leaf=1,
                                min_samples_split=2,
                                min_weight_fraction_leaf=0.0, n_estimators=100,
                                n_jobs=-1, oob_score=False,
                                power_transformer_method='box-cox',
                                power...
                                regressor=RandomForestRegressor(bootstrap=True,
                                                                ccp_alpha=0.0,
                                                                criterion='mse',
                           

In [ ]:
future_dates = pd.date_range(start = '2019-01-01', end = '2020-04-30', freq = 'W-MON')
#1st Jan 2019 to April 2020
   


In [ ]:
future_dates

DatetimeIndex(['2019-01-07', '2019-01-14', '2019-01-21', '2019-01-28',
               '2019-02-04', '2019-02-11', '2019-02-18', '2019-02-25',
               '2019-03-04', '2019-03-11', '2019-03-18', '2019-03-25',
               '2019-04-01', '2019-04-08', '2019-04-15', '2019-04-22',
               '2019-04-29', '2019-05-06', '2019-05-13', '2019-05-20',
               '2019-05-27', '2019-06-03', '2019-06-10', '2019-06-17',
               '2019-06-24', '2019-07-01', '2019-07-08', '2019-07-15',
               '2019-07-22', '2019-07-29', '2019-08-05', '2019-08-12',
               '2019-08-19', '2019-08-26', '2019-09-02', '2019-09-09',
               '2019-09-16', '2019-09-23', '2019-09-30', '2019-10-07',
               '2019-10-14', '2019-10-21', '2019-10-28', '2019-11-04',
               '2019-11-11', '2019-11-18', '2019-11-25', '2019-12-02',
               '2019-12-09', '2019-12-16', '2019-12-23', '2019-12-30',
               '2020-01-06', '2020-01-13', '2020-01-20', '2020-01-27',
      

In [ ]:
future_df = pd.DataFrame()



In [ ]:
future_df['month'] = [i.month for i in future_dates]
future_df['year'] = [i.year for i in future_dates] 
future_df['week'] = [i.week for i in future_dates] 

In [ ]:
df_train['Series'].tail()

808    809
809    810
810    811
811    812
812    813
Name: Series, dtype: int64

In [ ]:
len(future_dates)

69

In [ ]:
future_df['Series'] = list(range(814,883))

In [ ]:
future_df

,month,year,week,Series
0,1,2019,2,814
1,1,2019,3,815
2,1,2019,4,816
3,1,2019,5,817
4,2,2019,6,818
...,...,...,...,...
64,3,2020,14,878
65,4,2020,15,879
66,4,2020,16,880
67,4,2020,17,881


In [ ]:
predictions_future = predict_model(final_best, data=future_df)
predictions_future.head()

INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=PowerTransformedTargetRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                                max_depth=None, max_features='auto',
                                max_leaf_nodes=None, max_samples=None,
                                min_impurity_decrease=0.0,
                                min_impurity_split=None, min_samples_leaf=1,
                                min_samples_split=2,
                                min_weight_fraction_leaf=0.0, n_estimators=100,
                                n_jobs=-1, oob_score=False,
                                power_transformer_method='box-cox',
                                power...
                                regressor=RandomForestRegressor(bootstrap=True,
                                                                ccp_alpha=0.0,
                                                                criterion='mse',
                             

,month,year,week,Series,Label
0,1,2019,2,814,117.847794
1,1,2019,3,815,118.482053
2,1,2019,4,816,119.160391
3,1,2019,5,817,119.632926
4,2,2019,6,818,120.145866


In [ ]:
2019-04-28

In [ ]:
concat_df = pd.concat([df_train,predictions_future], axis=0)
concat_df_i = pd.date_range(start='2003-06-09', end = '2020-04-30', freq = 'W-MON')
concat_df.set_index(concat_df_i, inplace=True)
fig = px.line(concat_df, x=concat_df.index, y=["USD", "Label"], template = 'plotly_dark')
fig.show()

In [ ]:
save_model(final_best, 'model')

INFO:logs:Initializing save_model()
INFO:logs:save_model(model=PowerTransformedTargetRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                                max_depth=None, max_features='auto',
                                max_leaf_nodes=None, max_samples=None,
                                min_impurity_decrease=0.0,
                                min_impurity_split=None, min_samples_leaf=1,
                                min_samples_split=2,
                                min_weight_fraction_leaf=0.0, n_estimators=100,
                                n_jobs=-1, oob_score=False,
                                power_transformer_method='box-cox',
                                power...
                                regressor=RandomForestRegressor(bootstrap=True,
                                                                ccp_alpha=0.0,
                                                                criterion='mse',
                                       

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=['year', 'Series'],
                                       target='USD', time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numer...
                                                                                  ccp_alpha=0.0,
                                                                                  criterion='mse',
                                                                                  max_depth=None,
                  

In [ ]:
my_model = load_model('/content/model')

INFO:logs:Initializing load_model()
INFO:logs:load_model(model_name=/content/model, platform=None, authentication=None, verbose=True)


Transformation Pipeline and Model Successfully Loaded


In [ ]:
unseen_data = pd.read_csv('/content/sample_submission.csv')

In [ ]:
unseen_data['Date'] =pd.to_datetime(unseen_data['Date'])

In [ ]:
unseen_data.dropna(axis=1,inplace=True)

In [ ]:
unseen_data.head()

,Date,Prediction
0,2019-01-01,100
1,2019-01-02,100
2,2019-01-03,100
3,2019-01-04,100
4,2019-01-05,100


In [ ]:
unseen_data['month'] = [i.month for i in unseen_data['Date']]
unseen_data['year'] = [i.year for i in unseen_data['Date']]
unseen_data['week'] = [i.week for i in unseen_data['Date']]
unseen_data['Series'] = np.arange(1,len(unseen_data)+1)

In [ ]:
unseen_data.head()

,Date,Prediction,month,year,week,Series
0,2019-01-01,100,1,2019,1,1
1,2019-01-02,100,1,2019,1,2
2,2019-01-03,100,1,2019,1,3
3,2019-01-04,100,1,2019,1,4
4,2019-01-05,100,1,2019,1,5


In [ ]:
import sklearn
print(sklearn.__version__)

0.23.2


In [ ]:
forecasting = predict_model(my_model,unseen_data)

INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=[], ml_usecase='regression',
                                      numerical_features=['year', 'Series'],
                                      target='USD', time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numer...
                                                                                 ccp_alpha=0.0,
                                                                                 criterion='mse',
                                                         

In [ ]:
forecasting

,Date,Prediction,month,year,week,Series,Label
0,2019-01-01,100,1,2019,1,1,117.396901
1,2019-01-02,100,1,2019,1,2,117.396901
2,2019-01-03,100,1,2019,1,3,117.396901
3,2019-01-04,100,1,2019,1,4,117.396901
4,2019-01-05,100,1,2019,1,5,117.396901
5,2019-01-06,100,1,2019,1,6,117.396901
6,2019-01-07,100,1,2019,2,7,117.847794
7,2019-01-08,100,1,2019,2,8,117.847794
8,2019-01-09,100,1,2019,2,9,117.847794
9,2019-01-10,100,1,2019,2,10,117.847794


In [ ]:
pip install fastapi

In [ ]:
pip install uvicorn

In [ ]:
create_api(my_model,'petrol-forecast')

INFO:logs:Initializing get_config()
INFO:logs:get_config(variable=prep_pipe)
INFO:logs:Global variable: prep_pipe returned as Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=[], ml_usecase='regression',
                                      numerical_features=['year', 'Series'],
                                      target='USD', time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numer...
                ('scaling', 'passthrough'), ('P_transform', 'passthrough'),
                ('binn', 'passthrough'), ('rem_outliers', 'passthrough'),
                ('cluster_all', 'passthrough'),



API sucessfully created. This function only creates a POST API, it doesn't run it automatically.

To run your API, please run this command --> !python petrol-forecast.py
    


In [ ]:
!python petrol-forecast.py

Transformation Pipeline and Model Successfully Loaded
INFO:     Started server process [555]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Finished server process [555]
ERROR:    Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/starlette/routing.py", line 674, in lifespan
    await receive()
  File "/usr/local/lib/python3.8/dist-packages/uvicorn/lifespan/on.py", line 137, in receive
    return await self.receive_queue.get()
  File "/usr/lib/python3.8/asyncio/queues.py", line 163, in get
    await getter
asyncio.exceptions.CancelledError



In [ ]:
type(forecasting)

pandas.core.frame.DataFrame

In [ ]:
!python --version

Python 3.8.15
